In [1]:
import numpy as np
import os
import cv2
import torch

In [2]:
ds = "train"

In [3]:
test = torch.load(f"data/{ds}_dataset.pt")

In [4]:
imgs, labels = test.data, test.labels

In [5]:
import numpy as np

pairs = []
imgs = np.array(imgs)
labels = np.array(labels)

_imgs = []
_labels = []

for img, label in zip(imgs, labels):
    # Get indices of images with the same label and different labels
    same_class_indices = np.where(labels == label)[0]
    different_class_indices = np.where(labels != label)[0]

    if np.random.random() < 0.5:
        pair_index = np.random.choice(same_class_indices)
    else:
        pair_index = np.random.choice(different_class_indices)

    pair_img = imgs[pair_index]
    pair_label = labels[pair_index]

    _imgs.append((img, pair_img))
    _labels.append((label, pair_label))

    pairs.append((img, pair_img, label, pair_label))

In [6]:
np.save(f"{ds}_imgs.npy", _imgs)
np.save(f"{ds}_labels.npy", _labels)

In [7]:
from torch.utils.data import Dataset


class OpenSet(Dataset):
    def __init__(self, pairs, labels):
        self.pairs = pairs
        self.labels = labels

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, index):
        try:
            return self.pairs[index], self.labels[index]
        except IndexError as e:
            print(e)
            print(len(self.labels), index)

    def get_imgs(self):
        return self.pairs

    def get_n_classes(self):
        return len(set(self.labels))

    def labels_to_long(self):
        self.labels = self.labels.astype(np.int64)

    def get_class_data(self, label):
        return [self.pairs[i] for i, l in enumerate(self.labels) if l == label]

In [8]:
ps = [[pair[0], pair[1]] for pair in pairs]
ls = [[pair[2], pair[3]] for pair in pairs]
test_ds = OpenSet(ps, ls)

In [9]:
torch.save(test_ds, f"data/_{ds}_dataset.pt")

KeyboardInterrupt: 